In [4]:
import xml.etree.ElementTree as ET
import mysql.connector
from mysql.connector import errorcode

class XML:
    def __init__(self, archivoxml):
        self.tree = ET.parse(archivoxml)
        self.root = self.tree.getroot()
        self.archivoxml = archivoxml

    def limpieza_xml(self):
        def clean_gender(gender: str):
            if (gender == '0'):
                return 'Man'
            if (gender == '1'):
                return 'Woman'
            if (gender == '2'):
                return 'Nonbinary'
            if (gender == '3'):
                return 'Prefer not to say'
            if (gender == '4'):
                return 'Prefer to self-describe'

        for row in self.root.iter('row'):
            level_column = row.find('level_0')
            row.remove(level_column)
            gender_column = row.find('gender')
            gender_column.text = clean_gender(gender_column.text)
        

    def insertar_xml_sql(self):
        # Para evitar que nuestro código se pare utilizamos un try y except al hacer la conexión con
        # la base de datos de "project1".
        try:
            cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                          host='127.0.0.1',
                                          database='project1')
            print('Conexión con la base de datos realizada')
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print(
                    "Ha habido un error al introducir el nombre de usuario o la contraseña")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("La base de datos no existe")
            else:
                print(err)
        # Comprobamos que tenemos las tablas correctamente creadas en nuestra base de datos.
        # creamos el servidor
        cursor = cnx.cursor()
        # ejecutamos la query para ver las tablas de la BBDD que hemos indicado en la conexión
        cursor.execute("SHOW TABLES")
        print('---------------------------')
        print('Las tablas almacenadas en esta base de datos son:')
        for x in cursor:
            print(x)

        for a in self.tree.findall('row'):
            lista_xml1 = []

            time = a.find('time').text
            age = a.find('age').text
            gender = a.find('gender').text
            index_sql = a.find('index').text

            c = cnx.cursor()
            query = """INSERT INTO data_xml (time, age, gender, index_sql) VALUES (%s, %s, %s, %s)"""

            lista_xml1 = (time, age, gender, index_sql)
            lista_xml2 = [lista_xml1]

            try:
                for row in lista_xml2:
                    c.execute(query, row)
                    print("Caso", row, "registrado con éxito")
            except mysql.connector.Error as err:
                print('---------------------------')
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)

        cnx.commit()
        cursor.close()
        cnx.close()

        return
    pass


In [5]:
archivoxml = 'data_xml.xml'
xml = XML(archivoxml)
xml.limpieza_xml()
xml.insertar_xml_sql()

Conexión con la base de datos realizada
---------------------------
Las tablas almacenadas en esta base de datos son:
('data_sql',)
('data_txt',)
('data_xml',)
Caso ('784', '50-54', 'Man', '1') registrado con éxito
Caso ('924', '22-24', 'Man', '2') registrado con éxito
Caso ('575', '45-49', 'Man', '3') registrado con éxito
Caso ('781', '45-49', 'Man', '4') registrado con éxito
Caso ('1020', '25-29', 'Woman', '5') registrado con éxito
Caso ('141', '18-21', 'Woman', '6') registrado con éxito
Caso ('484', '30-34', 'Man', '7') registrado con éxito
Caso ('1744', '22-24', 'Man', '8') registrado con éxito
Caso ('655', '30-34', 'Man', '9') registrado con éxito
Caso ('1777', '40-44', 'Man', '10') registrado con éxito
Caso ('3081', '18-21', 'Woman', '11') registrado con éxito
Caso ('1922', '18-21', 'Woman', '12') registrado con éxito
Caso ('852', '45-49', 'Man', '13') registrado con éxito
Caso ('838', '22-24', 'Man', '14') registrado con éxito
Caso ('563', '35-39', 'Man', '15') registrado con éx